In [ ]:
import pandas as pd
from waymo_open_dataset import dataset_pb2 as open_dataset
import waymo.waymo_utils as wutils
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
from kitti import kitti_util as utils
import numpy as np
import plotly.graph_objects as go
import math

In [ ]:
row_idx = 5
pred_file_idx = 10
PREDS_PATH = '/root/workdir/waymo_challenge/predictions/'
PREDS_FILE = PREDS_PATH + f'pred_{pred_file_idx}.csv'
PREDS_PATH_2D = '/dataset/kitti_format/waymo/waymo_challenge_3/validation/preds_3/'
PREDS_FILE_2D = PREDS_PATH_2D + 'pred_all.csv'
TFRECORDS_PATH = '/dataset/waymo/validation/'

colors = [(255,0,0),
         (0,0,255),
         (0,255,0),
         (255,0,255),
         (0,255,255)]

df = pd.read_csv(PREDS_FILE) 
predictions_df_2d = pd.read_csv(PREDS_FILE_2D)

# Helper functions

In [ ]:
class Object3d(object):
    ''' 3d object label '''

    def __init__(self,obj):
        class_type = int(obj[1])
        x1,y1 = int(obj[2]), int(obj[3])
        x2,y2 = int(obj[4]), int(obj[5])
        tx, ty, tz = obj[6], obj[7], obj[8]
        h, w, l = obj[9], obj[10], obj[11]
        ry = obj[12]
        
        self.type = class_type  # 'Car', 'Pedestrian', ...
        self.truncation = 0  # truncated pixel ratio [0..1]
        self.occlusion = 0 # 0=visible, 1=partly occluded, 2=fully occluded, 3=unknown
        self.alpha = 0  # object observation angle [-pi..pi]

        # extract 2d bounding box in 0-based coordinates
        self.xmin = x1  # left
        self.ymin = y1  # top
        self.xmax = x2  # right
        self.ymax = y2  # bottom
        self.box2d = np.array([self.xmin, self.ymin, self.xmax, self.ymax])

        # extract 3d bounding box information
        self.h = h  # box height
        self.w = w # box width
        self.l = l # box length (in meters)
        self.t = (tx, ty, tz)  # location (x,y,z) in camera coord.
        self.ry = ry  # yaw angle (around Y-axis in camera coordinates) [-pi..pi]

        self.score = 1

def get_box_transformation_matrix_my(obj_loc, obj_size, ry):
    """Create a transformation matrix for a given label box pose."""

    tx,ty,tz = obj_loc
    c = math.cos(ry)
    s = math.sin(ry)

    sl, sh, sw = obj_size

    return np.array([
        [       -sl*c,    0,  sw*s,    tx],
        [          0,  -sh,    0,      ty],
        [    -sl*(-s),    0,  sw*c,    tz],
        [          0,    0,    0,     1]])

def visualize_2d_predictions(predictions_df_2d, frame_img_name, image, confidence_thresh_2d=0.5):
    pred_df_row = predictions_df_2d.loc[predictions_df_2d['image_name'] == frame_img_name]
    bboxes = pred_df_row['bboxes'].iloc[0].split(',')
    bboxes_float = [[float(bbox_el) for bbox_el in bbox[1:-1].split(' ')] for bbox in bboxes]

    for bbox in bboxes_float:
        center_x, center_y, w, h, pred_class, score = bbox
        if score >= confidence_thresh_2d:
            x1, y1, x2, y2 = int(center_x - w/2), int(center_y - h/2), int(center_x + w/2), int(center_y + h/2)
            cv2.rectangle(
                    image,
                    (x1, y1),
                    (x2, y2),
                    colors[int(pred_class)],
                    2,
                )

    fig=plt.figure(figsize=(32, 16))
    plt.imshow(image)
    
def visualize_2d_frustum(image, objects_in_cam):
    for obj in objects_in_cam:
        cv2.rectangle(
                image,
                (int(obj[2]), int(obj[3])),
                (int(obj[4]), int(obj[5])),
                colors[int(obj[1])],
                2,
            )

    fig=plt.figure(figsize=(32, 16))
    plt.imshow(image)
    
def visualize_3d_frustum(image, calib, objects_in_cam, filter_class=[0,1,2,3]):
    objects_in_cam = objects_by_cam_id[cam_idx]

    for obj in objects_in_cam:
        obj_t = Object3d(obj)
        if obj_t.type in filter_class:
            box3d_pts_2d, box3d_pts_3d = utils.compute_box_3d(obj_t, calib.P)
            if box3d_pts_2d is None or box3d_pts_3d is None:
                print('Skip box')
                continue
            image = utils.draw_projected_box3d(image, box3d_pts_2d, color=colors[obj_t.type])

    fig=plt.figure(figsize=(32, 16))
    plt.imshow(image)
    
def show_interactive_lidar(lidar_scan, calib=None, objects=[], with_boxes=False,to_show=True):
    
    Tr_V2C = np.eye(4)
    Tr_V2C[:3,:] = calib.V2C
    lidar_scan = np.matmul(Tr_V2C, lidar_scan.T).T
    
    distances = np.sqrt(np.sum(lidar_scan**2, axis=1))
    scatter = go.Scatter3d(x=lidar_scan[:,2], y=-lidar_scan[:,0], z=-lidar_scan[:,1],
                                   mode='markers',marker=dict(size=2,                # set color to an array/list of desired values
                                                              color=distances,
                                                              colorscale='Viridis',   # choose a colorscale
                                                              opacity=0.8
                                                            ))
    if with_boxes:
        x_lines = []
        y_lines = []
        z_lines = []

        def f_lines_add_nones():
            x_lines.append(None)
            y_lines.append(None)
            z_lines.append(None)

        ixs_box_0 = [0, 1, 2, 3, 0]
        ixs_box_1 = [4, 5, 6, 7, 4]
        axes_transformation = np.array([[0,-1,0,0],
                                        [0,0,-1,0],
                                        [1,0,0,0],
                                        [0,0,0,1]])
        axes_transformation_inv = np.linalg.inv(axes_transformation)
        for obj_i in objects:
            obj = Object3d(obj_i)
#             points = view_points(box.corners(), view=np.eye(3), normalize=False)
            Tr_2box = get_box_transformation_matrix_my(obj.t, (obj.l, obj.h, obj.w), obj.ry)
#             Tr_2box = np.matmul(Tr_2box,axes_transformation)

            x_corners =  np.array([1/2, -1/2, -1/2, 1/2, 1/2, -1/2, -1/2, 1/2])
            y_corners =  np.array([1, 1, 0, 0, 1, 1, 0, 0])
            z_corners =  np.array([1/2, 1/2, 1/2, 1/2, -1/2, -1/2, -1/2, -1/2])            
            
            corners = np.vstack((x_corners, y_corners, z_corners, np.ones(8)))
            points = np.matmul(Tr_2box, corners)
            
            x_lines.extend(points[0, ixs_box_0])
            y_lines.extend(points[1, ixs_box_0])
            z_lines.extend(points[2, ixs_box_0])
            f_lines_add_nones()
            x_lines.extend(points[0, ixs_box_1])
            y_lines.extend(points[1, ixs_box_1])
            z_lines.extend(points[2, ixs_box_1])
            f_lines_add_nones()
            for i in range(4):
                x_lines.extend(points[0, [ixs_box_0[i], ixs_box_1[i]]])
                y_lines.extend(points[1, [ixs_box_0[i], ixs_box_1[i]]])
                z_lines.extend(points[2, [ixs_box_0[i], ixs_box_1[i]]])
                f_lines_add_nones()
        y_lines = [-1*val if val is not None else val for val in y_lines]
        x_lines = [-1*val if val is not None else val for val in x_lines]
        lines = go.Scatter3d(x=z_lines, y=x_lines, z=y_lines, mode="lines", name="lines")
        fig = go.Figure(data=[scatter, lines])
    else:
        fig = go.Figure(data=[scatter])
    fig.update_layout(scene_aspectmode="data")
    if to_show:
        fig.show()

In [ ]:
frame = df.iloc[row_idx]

tfrecord_name = frame['tfrecord_name']
tfrecord_path = TFRECORDS_PATH + tfrecord_name
tfrec_idx = frame['tfrec_idx']
frame_idx = frame['frame_idx']
predictions = frame['predictions']

print(predictions)
preds = predictions.split(',')
objects = [[float(pred_el) for pred_el in pred[1:-1].split(' ')] for pred in preds]

objects_by_cam_id = {1: [],
                     2: [],
                     3: [],
                     4: [],
                     5: []}
for obj in objects:
    objects_by_cam_id[int(obj[0])].append(obj)

In [ ]:
dataset = tf.data.TFRecordDataset(tfrecord_path, compression_type='')
frame = open_dataset.Frame()

for idx, data_i in enumerate(dataset):
    if idx == frame_idx:     
        frame.ParseFromString(bytearray(data_i.numpy()))

images = wutils.get_images(frame)
calibs = wutils.get_calibs(frame)
pc = wutils.get_lidar(frame)

# Visualization

In [ ]:
CONFIDENCE_THRESH_2D = 0.5
cam_idx = 1

## 2D Predictions from mmdetection

In [ ]:
frame_img_name = f'{tfrec_idx}_{frame_idx}_{cam_idx}.jpg'
image = images[cam_idx-1].copy()
visualize_2d_predictions(predictions_df_2d, frame_img_name, image, confidence_thresh_2d=CONFIDENCE_THRESH_2D)

## Visualize frustum 2d

In [ ]:
image = images[cam_idx-1].copy()
objects_in_cam = objects_by_cam_id[cam_idx]

visualize_2d_frustum(image, objects_in_cam)

## Visualize frustum 3D on image

In [ ]:
image = images[cam_idx-1].copy()
calib = calibs[cam_idx-1]
objects_in_cam = objects_by_cam_id[cam_idx]

visualize_3d_frustum(image, calib, objects_in_cam, filter_class=[0,1])

## Visualize frustum 3D on pointcloud

In [ ]:
show_interactive_lidar(pc, calib=calibs[cam_idx-1], objects=objects_by_cam_id[cam_idx], with_boxes=True,to_show=True)

# Cut region from pointcloud based on 2d detection

In [ ]:
from waymo.prepare_data_waymo import extract_frustum_data_inference
from datasets.dataset_info import WaymoCategory
from datasets.data_utils import rotate_pc_along_y, project_image_to_rect, compute_box_3d, extract_pc_in_box3d, roty
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision
import open3d as o3d

def generate_ref(box, P):
    s1, s2, s3, s4 = (0.1, 0.2, 0.4, 0.8)

    z1 = np.arange(0, 70, s1) + s1 / 2.
    z2 = np.arange(0, 70, s2) + s2 / 2.
    z3 = np.arange(0, 70, s3) + s3 / 2.
    z4 = np.arange(0, 70, s4) + s4 / 2.

    cx, cy = (box[0] + box[2]) / 2., (box[1] + box[3]) / 2.,

    xyz1 = np.zeros((len(z1), 3))
    xyz1[:, 0] = cx
    xyz1[:, 1] = cy
    xyz1[:, 2] = z1
    xyz1_rect = project_image_to_rect(xyz1, P)

    xyz2 = np.zeros((len(z2), 3))
    xyz2[:, 0] = cx
    xyz2[:, 1] = cy
    xyz2[:, 2] = z2
    xyz2_rect = project_image_to_rect(xyz2, P)

    xyz3 = np.zeros((len(z3), 3))
    xyz3[:, 0] = cx
    xyz3[:, 1] = cy
    xyz3[:, 2] = z3
    xyz3_rect = project_image_to_rect(xyz3, P)

    xyz4 = np.zeros((len(z4), 3))
    xyz4[:, 0] = cx
    xyz4[:, 1] = cy
    xyz4[:, 2] = z4
    xyz4_rect = project_image_to_rect(xyz4, P)

    return xyz1_rect, xyz2_rect, xyz3_rect, xyz4_rect


def get_center_view_rot_angle(frustum_angle):
    ''' Get the frustum rotation angle, it isshifted by pi/2 so that it
    can be directly used to adjust GT heading angle '''
    return np.pi / 2.0 + frustum_angle


def get_box3d_center(box3d):
    ''' Get the center (XYZ) of 3D bounding box. '''
    box3d_center = (box3d[0, :] + box3d[6, :]) / 2.0
    return box3d_center


def get_center_view_box3d_center(box3d, frustum_angle):
    ''' Frustum rotation of 3D bounding box center. '''
    box3d_center = (box3d[0, :] + box3d[6, :]) / 2.0
    return rotate_pc_along_y(np.expand_dims(box3d_center, 0), get_center_view_rot_angle(frustum_angle)).squeeze()


def get_center_view_box3d(box3d, frustum_angle):
    ''' Frustum rotation of 3D bounding box corners. '''
    box3d = box3d
    box3d_center_view = np.copy(box3d)
    return rotate_pc_along_y(box3d_center_view, get_center_view_rot_angle(frustum_angle))


def get_center_view_point_set(input_i, frustum_angle):
    ''' Frustum rotation of point clouds.
    NxC points with first 3 channels as XYZ
    z is facing forward, x is left ward, y is downward
    '''
    # Use np.copy to avoid corrupting original data
    point_set = np.copy(input_i)
    return rotate_pc_along_y(point_set, get_center_view_rot_angle(frustum_angle))


def get_center_view(point_set, frustum_angle):
    ''' Frustum rotation of point clouds.
    NxC points with first 3 channels as XYZ
    z is facing forward, x is left ward, y is downward
    '''
    # Use np.copy to avoid corrupting original data
    point_set = np.copy(point_set)
    return rotate_pc_along_y(point_set, get_center_view_rot_angle(frustum_angle))


def get_frustum_input(data,  one_hot=True, npoints=1024, min_n_points = 5):
    box2d_i = data['box2d_i']
    input_i = data['input_i']
    type_i = data['type_i']
    frustum_angle_i = data['frustum_angle_i']
    calib_i = data['calib_i']

    rotate_to_center = True
    with_extra_feat = False

    ''' Get index-th element from the picked file dataset. '''
    # ------------------------------ INPUTS ----------------------------
    rot_angle = get_center_view_rot_angle(frustum_angle_i )

    cls_type = type_i
    assert cls_type in WaymoCategory.CLASSES, cls_type
    size_class = WaymoCategory.CLASSES.index(cls_type)

    # Compute one hot vector
    if one_hot:
        one_hot_vec = np.zeros((3))
        one_hot_vec[size_class] = 1

    # Get point cloud
    if rotate_to_center:
        point_set = get_center_view_point_set(input_i, frustum_angle_i)
    else:
        point_set = input_i 

    if not with_extra_feat:
        point_set = point_set[:, :3]

    if point_set.shape[0]<= min_n_points:
        # print(f'No enougth number of points {point_set.shape[0]}')
        return None
    # Resample
    if npoints > 0:
        choice = np.random.choice(point_set.shape[0], npoints, point_set.shape[0] < npoints)
    else:
        choice = np.random.permutation(len(point_set.shape[0]))

    point_set = point_set[choice, :]

    box = box2d_i
    P = calib_i.P

    ref1, ref2, ref3, ref4 = generate_ref(box, P)

    if rotate_to_center:
        ref1 = get_center_view(ref1, frustum_angle_i)
        ref2 = get_center_view(ref2, frustum_angle_i)
        ref3 = get_center_view(ref3, frustum_angle_i)
        ref4 = get_center_view(ref4, frustum_angle_i)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(ref1)
    o3d.io.write_point_cloud("xyz1.pcd", pcd)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(ref2)
    o3d.io.write_point_cloud("xyz2.pcd", pcd)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(ref3)
    o3d.io.write_point_cloud("xyz3.pcd", pcd)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(ref4)
    o3d.io.write_point_cloud("xyz4.pcd", pcd)
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_set)
    o3d.io.write_point_cloud("pedestrian.pcd", pcd)

    data_inputs = {
        'point_cloud': torch.FloatTensor(point_set).transpose(1, 0),
        'rot_angle': torch.FloatTensor([rot_angle]),
        'center_ref1': torch.FloatTensor(ref1).transpose(1, 0),
        'center_ref2': torch.FloatTensor(ref2).transpose(1, 0),
        'center_ref3': torch.FloatTensor(ref3).transpose(1, 0),
        'center_ref4': torch.FloatTensor(ref4).transpose(1, 0),
    }

    if not rotate_to_center:
        data_inputs.update({'rot_angle': torch.zeros(1)})

    if one_hot:
        data_inputs.update({'one_hot': torch.FloatTensor(one_hot_vec)})
    
    return data_inputs

frame_img_name = f'{tfrec_idx}_{frame_idx}_{cam_idx}.jpg'
pred_df_row = predictions_df_2d.loc[predictions_df_2d['image_name'] == frame_img_name]
bboxes = pred_df_row['bboxes'].iloc[0].split(',')
bboxes_float = [[float(bbox_el) for bbox_el in bbox[1:-1].split(' ')] for bbox in bboxes]
image = images[cam_idx-1].copy()

for bbox in bboxes_float:
        center_x, center_y, w, h, pred_class, score = bbox
        if score >= CONFIDENCE_THRESH_2D and pred_class==2:
            x1, y1, x2, y2 = int(center_x - w/2), int(center_y - h/2), int(center_x + w/2), int(center_y + h/2)
            data = extract_frustum_data_inference([x1, y1, x2, y2, 'PEDESTRIAN', 1], pc, images[cam_idx-1], calibs[cam_idx-1], type_whitelist=['PEDESTRIAN'])
            data_dict = get_frustum_input(data)
            break
